---
<center><h1> Lesson 8.1 - Work with Twitter API in Python</center></h1>

---

A Web API is an *application programming interface* (**API**). It helps easely to entaract with program programs and web services.  

Today many services provide its API for developers (such as Facebook, Google, Twitter, etc). API helps to en to interact with their services and to access data in a programmatic way. With help of API, you can create your own application.  

API is a set of subroutine definitions, protocols, and tools for building application software. In general terms, it is a set of clearly defined methods of communication between various software components. A good API makes it easier to develop a computer program by providing all the building blocks, which are then put together by the programmer. An API may be for a web-based system, operating system, database system, computer hardware or software library.  

The current lesson is devoted to Twitter API. We will consider both approaches of getting access to Twitter API - main Python tools such as `request` library and specific Python libraries such as `tweepy`.

<img src="images/Python_Twitter.jpg" width="512px" height="288px">

One of the reasons that Twitter is so popular, and become what it is today, is due to the huge number of developers who have built apps over the years. Why so many developers? Basically because Twitter has been very easy to work with. Twitter uses the OAuth protocol for secure application development. 

In this lesson, you will create an application integrated with Twitter. Registering the application is the first step in developing an application that is integrated into its users' social graph.

## Register a new application
All Twitter users are potentially Twitter application developers. Visit https://apps.twitter.com and and press "Create New App".   

<img src="images/twitter_01.jpg">

After that fill Aplication Detals (Name, Description and Website), read and agree to the Twitter Developer Agreement. Press "Create your Twitter application".  
(Note that you must add your mobile phone to your Twitter profile before creating an application. Please read [here](https://support.twitter.com/articles/110250-adding-your-mobile-number-to-your-account-via-web) for more information.)

<img src="images/twitter_02.jpg">  
After Captcha challenge click "Create your Twitter application" to complete the form and go to the application settings page.

We now need to authorise the Twitter app for your Twitter account. To do this you should select "Key and Access Tokens" menu button and create your access token. This **access token** will allow your Twitter application to read Twitter information. You’ll be able to get data of your tweets, mentions, lists and more.  

<img src="images/twitter_03.jpg">

In the opened window you’ll then be presented with lots of information. The main fields to note are **_Consumer key_** and **_Consumer secret_**. These values are your application’s credentials for Twitter. You need them to do almost anything with Twitter, including going through the OAuth authorization flow and working with Twitter’s REST API. But we’re not quite done yet. 

As a result you will see the window with consumer and token keys for your application 

<img src="images/twitter_04.jpg">

Above the four fields are highlighted. You will need these long horrible strings of characters for your Twitter app. 

---
## 1. Getting access to Twitter APIs using main Python tools
---

Many web services including Twitter require authentication, and there are many different types. 

Python library `requests` provides a pythonic way to make complex HTTP requests, and handles difficult tasks like authentication.

A common form of authentication for several web APIs is OAuth. The `requests-oauthlib` library allows `requests`'s users to easily make OAuth authenticated requests.

In [1]:
import requests
from requests_oauthlib import OAuth1

import json

# Let's define consumer and access keys and secrets for getting access to Twitter API through your application
consumer_key = 'o7ApzFDk58YKYoCXR7au2Qa8o'
consumer_secret = 'sCruk5UCcQpuyL7NWPizcgfDHeJEG5CEioG43j6iNOmUymJsVa'

access_token = '1437963396-DdboBDabBE3KyUqHd8Zz3sr8MxVpKDwVRJRUjiC'
access_secret = 'R4GmIhhjx2doAZFam1EMrgFuxFe5u7zXRBR7dLEgGjqSp'

# You will authenticate yourself using OAuth1 object
auth = OAuth1(consumer_key, consumer_secret, access_token, access_secret)
print auth

In [2]:
# If the authentication was successful, you should see the name of the account print out
url_0 = 'https://api.twitter.com/1.1/account/verify_credentials.json'
res = requests.get(url_0, auth=auth)

print "My name is", res.json()['name'] 

My name is Dmitriy Kisil


### 1.1: Read Tweets from Your Homepage

The list of all available Twitter API you can find here https://developer.twitter.com/en/docs/api-reference-index. Let's use `home_timeline` that returns a collection of the most recent tweets and retweets posted by the authenticating user and the users they follow.

In [3]:
url_1 = 'https://api.twitter.com/1.1/statuses/home_timeline.json'
res = requests.get(url_1, auth=auth, params={"count": 3})

# The res object encapsulates the "response" of the server. Notice the status code that is displayed. 
# Code 200 means that things went fine
# Code 403 means that the server understood the request, but is refusing to fulfill it
# Code 404 means that the URL was not found
# Codes 5xx mean that something went wrong

print res, "Status code:", res.status_code
# Let's see how looks the url
print res.url
print "Content type:", res.headers['content-type'], '\n'

tweets = res.json()
if res.status_code == 200:
    for tweet in tweets:
        print tweet['text']
else:   # You have no tweets
    print tweets

<Response [200]> Status code: 200
https://api.twitter.com/1.1/statuses/home_timeline.json?count=3
Content type: application/json;charset=utf-8 

RT @PleaseBeGneiss: Stranger asks you what time it is = kinda annoying

Stranger asks you what year it is = pretty concerning 

Stranger as…
RT @AshleyS89102847: I'm beyond excited and proud to say I did my first coding project with @freeCodeCamp!!!! The feeling of accomplishment…
PM дайджест #16: разбираемся с OKR, как вернуть Scrum-команде вдохновение, метрики в разработке… https://t.co/t6QrF63bTU


### 1.2: Search tweets by key words

Twitter employs a special query language.  For example, the query _"luck?"_ will return tweets that contain the word "luck" and are phrased as a question OR the query _"winter"_ will return the word "winter" with a positive attitude. [Check out more examples here](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets).

We can mine tweets using either **_search_** or **_stream_**.

>The key difference between **_stream_** and **_search_** is that **_stream_** provides new data as it comes in, while **_search_** can be used to query old data. The **_search_** API is more powerful for queries, and provides faster access to a wide-range of data.

Let's search for a single tweet about "luck?", phrased as a question. Note if request phrase contains URI characters you should [encode](https://en.wikipedia.org/wiki/Percent-encoding) these queries before making the request, because these characters can play role of keys in an URL. Particularly, the question mark "?" is encoded as "%3F".

In [4]:
url_2 = 'https://api.twitter.com/1.1/search/tweets.json?q=luck%3F'
res = requests.get(url_2, auth=auth)

print res, res.status_code, res.headers['content-type']
print res.url

tweets = res.json()
tweets

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/search/tweets.json?q=luck%3F


{u'search_metadata': {u'completed_in': 0.043,
  u'count': 15,
  u'max_id': 1089114579689398272,
  u'max_id_str': u'1089114579689398272',
  u'next_results': u'?max_id=1089114254366584831&q=luck%3F&include_entities=1',
  u'query': u'luck%3F',
  u'refresh_url': u'?since_id=1089114579689398272&q=luck%3F&include_entities=1',
  u'since_id': 0,
  u'since_id_str': u'0'},
 u'statuses': [{u'contributors': None,
   u'coordinates': None,
   u'created_at': u'Sat Jan 26 10:55:25 +0000 2019',
   u'entities': {u'hashtags': [],
    u'symbols': [],
    u'urls': [],
    u'user_mentions': [{u'id': 765224958138077184,
      u'id_str': u'765224958138077184',
      u'indices': [0, 10],
      u'name': u'Vulva Whisperer',
      u'screen_name': u'brobro___'}]},
   u'favorite_count': 0,
   u'favorited': False,
   u'geo': None,
   u'id': 1089114579689398272,
   u'id_str': u'1089114579689398272',
   u'in_reply_to_screen_name': u'brobro___',
   u'in_reply_to_status_id': 1089111965899714560,
   u'in_reply_to_status_

Please pay your attention how many information each tweet contains.

In [5]:
# And let's see tweets text directly
for num, tweet in enumerate(tweets['statuses']):
    print 'Tweet #{0}\t{1}\n{2}\n'.format(num+1, tweet['created_at'], tweet['text'].encode("utf-8"))

Tweet #1	Sat Jan 26 10:55:25 +0000 2019
@brobro___ oh, ini kamu yang sekarang, pribadi yang merasa lebih hanya karna punya uang? good luck ya :)

Tweet #2	Sat Jan 26 10:55:23 +0000 2019
RT @Luljongin88: GIVEAWAY INA ONLY📣📣📣

Aku adain giveaway ini karena Januari ngepasin Kyungsoo, Jongin dan Aku ulangtahun nih.... Dan ada r…

Tweet #3	Sat Jan 26 10:55:19 +0000 2019
RT @Luljongin88: GIVEAWAY INA ONLY📣📣📣

Aku adain giveaway ini karena Januari ngepasin Kyungsoo, Jongin dan Aku ulangtahun nih.... Dan ada r…

Tweet #4	Sat Jan 26 10:55:11 +0000 2019
RT @Luljongin88: GIVEAWAY INA ONLY📣📣📣

Aku adain giveaway ini karena Januari ngepasin Kyungsoo, Jongin dan Aku ulangtahun nih.... Dan ada r…

Tweet #5	Sat Jan 26 10:55:08 +0000 2019
RT @topislamictwit: "Gantikan ucapan good luck kepada."

"Allah yuftah alaikum."

"Maksudnya?"

"Semoga Allah membuka jalan kepada kamu."…

Tweet #6	Sat Jan 26 10:55:04 +0000 2019
RT @FashionNova: 🚨Contest🚨 Do you want to win a $200 FN gift card?! 🤑 🛍🙌

Here's how:

1

You may create very specialized and more concrete queries. 

Let's find 5 tweets that contains the word "python" or "IPython" near London. We can provide this as a `geocode` with a lattitude, longitude and radius. We can also specify time range of tweets appearance (with the help of `since` and `until` key words) and the tweet language (the `lang` parameter restricts tweets to the given language). Additional parameters allows build very complicated search requests.

In [6]:
url_2_2 = 'https://api.twitter.com/1.1/search/tweets.json?'

params = {
    "q": 'python OR IPython',
    "count": 5, 
    "show_user": False,
    "geocode": '51.509865,-0.118092,20mi',
    "lang": 'en'
}

res = requests.get(url_2_2, auth=auth, params=params)

print res, res.status_code, res.headers['content-type']
print res.url

tweets = res.json()
for num, tweet in enumerate(tweets['statuses']):
    print 'Tweet #{0}\t{1}\n{2}\n'.format(num+1, tweet['created_at'], tweet['text'].encode("utf-8"))

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/search/tweets.json?q=python+OR+IPython&count=5&geocode=51.509865%2C-0.118092%2C20mi&lang=en&show_user=False
Tweet #1	Sat Jan 26 10:55:37 +0000 2019
RT @BAFTA: We're here to sprinkle a bit of fun over your weekend as we look back at Monty Python's hilarious Outstanding Contribution To Br…

Tweet #2	Sat Jan 26 10:53:05 +0000 2019
RT @RyoIndie: Making a game isn't easy. Composing is a key part of it! #composing #coding #python #pygame #programming #sounddesign #music…

Tweet #3	Sat Jan 26 10:52:20 +0000 2019
RT @LoomiAssistant: Old vs New (well old with #AI)

#Machinelearning #java #python #ArtificialIntelligence 
#programming #DataScience @Kirk…

Tweet #4	Sat Jan 26 10:45:28 +0000 2019
RT @BAFTA: We're here to sprinkle a bit of fun over your weekend as we look back at Monty Python's hilarious Outstanding Contribution To Br…

Tweet #5	Sat Jan 26 10:33:06 +0000 2019
Just like a Monty Python sketch.. or should t

### 1.3: Get a list of all your followers

Twitter GET request `friends/list` returns a cursored collection of user objects for every user the specified user is following (otherwise known as their “friends”).

In [7]:
url_3 = 'https://api.twitter.com/1.1/friends/list.json'
res = requests.get(url_3, auth=auth)

print res, res.status_code, res.headers['content-type']
print res.url

friends = res.json()
friends

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/friends/list.json


{u'next_cursor': 1590300932417656656,
 u'next_cursor_str': u'1590300932417656656',
 u'previous_cursor': 0,
 u'previous_cursor_str': u'0',
 u'total_count': None,
 u'users': [{u'blocked_by': False,
   u'blocking': False,
   u'contributors_enabled': False,
   u'created_at': u'Wed Jan 10 23:40:13 +0000 2018',
   u'default_profile': False,
   u'default_profile_image': False,
   u'description': u'Democratizing AI, one meme at the time! OC unless specified otherwise. Ran by @Aleksa_C_',
   u'entities': {u'description': {u'urls': []},
    u'url': {u'urls': [{u'display_url': u'facebook.com/artificialinte\u2026',
       u'expanded_url': u'https://www.facebook.com/artificialintelligencememes/',
       u'indices': [0, 23],
       u'url': u'https://t.co/OdUkErt5Tf'}]}},
   u'favourites_count': 320,
   u'follow_request_sent': False,
   u'followers_count': 5224,
   u'following': True,
   u'friends_count': 24,
   u'geo_enabled': False,
   u'has_extended_profile': False,
   u'id': 951237270044069888,
 

In [10]:
# And let's see only friends' name
for num, friends in enumerate(friends['users']):
    print 'My friend #{0}\t{1}'.format(num+1, friends['name'].encode("utf-8"))

My friend #1	AI Memes for Artificially Intelligent Teens
My friend #2	John Papa
My friend #3	Twitter
My friend #4	Rasmus Bååth
My friend #5	The Lazy Goldmaker
My friend #6	Scott McMillan
My friend #7	Method
My friend #8	Ned Batchelder
My friend #9	Patrick Shyu
My friend #10	Kent Davis
My friend #11	angela he
My friend #12	Mikel Bober-Irizar
My friend #13	Ben Brode
My friend #14	+380991741601
My friend #15	WACKO
My friend #16	Andrej Karpathy
My friend #17	Chad Gardner
My friend #18	Tom Augspurger
My friend #19	Wes McKinney
My friend #20	Inside Amazon


### 1.4:  Finding what are in the trend somewhere

We can return the top 50 trending topics for a specific location, where the location is a `WOEID (Yahoo Where on Earth ID)`. 

The `WOEID` is a unique identifier, similar to zipcodes, but that expand worldwide. For example, London has a `WOEID` of 44418. You can search for `WOEID`'s here: http://woeid.rosselliot.co.nz/.

Let's see the top trending topics in London

In [9]:
url_4 = 'https://api.twitter.com/1.1/trends/place.json?id=44418'
res = requests.get(url_4, auth=auth)

print res, res.status_code, res.headers['content-type']
print res.url

top50_trends = res.json()
top50_trends

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/trends/place.json?id=44418


[{u'as_of': u'2019-01-26T10:56:44Z',
  u'created_at': u'2019-01-26T10:51:50Z',
  u'locations': [{u'name': u'London', u'woeid': 44418}],
  u'trends': [{u'name': u'#Caturday',
    u'promoted_content': None,
    u'query': u'%23Caturday',
    u'tweet_volume': None,
    u'url': u'http://twitter.com/search?q=%23Caturday'},
   {u'name': u'Osaka',
    u'promoted_content': None,
    u'query': u'Osaka',
    u'tweet_volume': 70775,
    u'url': u'http://twitter.com/search?q=Osaka'},
   {u'name': u'#SaturdayMorning',
    u'promoted_content': None,
    u'query': u'%23SaturdayMorning',
    u'tweet_volume': None,
    u'url': u'http://twitter.com/search?q=%23SaturdayMorning'},
   {u'name': u'#AustraliaDay',
    u'promoted_content': None,
    u'query': u'%23AustraliaDay',
    u'tweet_volume': 20529,
    u'url': u'http://twitter.com/search?q=%23AustraliaDay'},
   {u'name': u'Kvitova',
    u'promoted_content': None,
    u'query': u'Kvitova',
    u'tweet_volume': 22267,
    u'url': u'http://twitter.com/sea

In [10]:
# And let's see only the first 10 trends names and the respective URL
for i, trend in enumerate(top50_trends[0]['trends'][:10]):
    print '{0} - {1} - URL: {2}'.format(i+1, trend['name'], trend['url'])

1 - #Caturday - URL: http://twitter.com/search?q=%23Caturday
2 - Osaka - URL: http://twitter.com/search?q=Osaka
3 - #SaturdayMorning - URL: http://twitter.com/search?q=%23SaturdayMorning
4 - #AustraliaDay - URL: http://twitter.com/search?q=%23AustraliaDay
5 - Kvitova - URL: http://twitter.com/search?q=Kvitova
6 - #BigGardenBirdWatch - URL: http://twitter.com/search?q=%23BigGardenBirdWatch
7 - #SaturdayMotivation - URL: http://twitter.com/search?q=%23SaturdayMotivation
8 - GAME DAY - URL: http://twitter.com/search?q=%22GAME+DAY%22
9 - Soccer AM - URL: http://twitter.com/search?q=%22Soccer+AM%22
10 - Michel Legrand - URL: http://twitter.com/search?q=%22Michel+Legrand%22


### 1.5: Streaming 

Twitter offers a [Streaming API](https://dev.twitter.com/streaming/overview) to make it easier to query streams of tweets.  The Stream API encapsulates some pain points of REST access to ensure that Stream calls don't exceed the rate limit. These tool allows to get tweet data when it appears in real time. There are three stream types:

- `Public Streams:` Streams of public data flowthing through Twitter. Suitable for followign specific users, topics or for data mining.
- `User Streams:` Single-user streams. Containing roughly all of the data corresponding with a single user's view of Twitter. 
- `Site Streams:` The multi-user version of user streams.

>Connecting to the streaming API requires keeping a persistent HTTP connection open. In many cases this involves thinking about your application differently than if you were interacting with the REST API. An app which connects to the Streaming APIs will not be able to establish a connection in response to a user request. Instead, the code for maintaining the Streaming connection is typically run in a process separate from the process which handles HTTP requests. The streaming process gets the input tweets and performs any parsing, filtering, and/or aggregation needed before storing the result to a data store. The HTTP handling process queries the data store for results in response to user requests. While this model is more complex than the first example, the benefits from having a realtime stream of tweet data make the integration worthwhile for many types of apps.

With `requests.Response.iter_lines()` you can easily iterate over streaming APIs including the **Twitter Streaming API**. Simply set `stream` parameter to `True` and iterate over the response with `iter_lines()`. 

Depending on the search term/terms, we can get tons of tweets within a few minutes.

A working example that gathers all the new tweets with the _#android_ hashtag:

In [13]:
#This cell doesn't have limits and doesn't stop automatically. You need to interrupt it by yourself.
import time

url_5 = 'https://stream.twitter.com/1.1/statuses/filter.json?track=android'
res = requests.get(url_5, auth=auth, stream=True)

print res, res.status_code, res.headers['content-type']
print res.url, "\n"

# Let's measure the time elapsed after the start of streaming 
start = time.time()
for line in res.iter_lines():
    # filter out keep-alive new lines
    if line:
        parsed_line = json.loads(line)
        print "Elapsed:", time.time()-start, "sec", parsed_line["created_at"], "\n", parsed_line['text'].encode("utf-8"), "\n"

<Response [200]> 200 application/json
https://stream.twitter.com/1.1/statuses/filter.json?track=android 

Elapsed: 0.870567083359 sec Fri Jan 25 16:27:35 +0000 2019 
Orion Labs Inc. Redesigns Its Push-to-Talk App on Android https://t.co/TDRQT8jEXc #news https://t.co/b4jBWRqJSU 

Elapsed: 1.0116751194 sec Fri Jan 25 16:27:35 +0000 2019 
モイ！Androidからキャス配信中 - https://t.co/KA8WUlstql 

Elapsed: 1.99296498299 sec Fri Jan 25 16:27:36 +0000 2019 
モイ！Androidからキャス配信中 - / 京都８レースだけ https://t.co/yCDhGmmAVk 

Elapsed: 2.45216798782 sec Fri Jan 25 16:27:36 +0000 2019 
@flowerpathtae Y'all flexing your iPhone lock screens and wallpapers is giving my cheap android ass some major complexes (≖_≖ ) 

Elapsed: 2.63300800323 sec Fri Jan 25 16:27:36 +0000 2019 
Mobiwol :  Τείχος προστασίας για συσκευές Android https://t.co/P634uQv0fu 

Elapsed: 3.37269711494 sec Fri Jan 25 16:27:37 +0000 2019 
RT @jacksfilms: Twitter for Android https://t.co/TXoQnUFkAB 

Elapsed: 3.97668600082 sec Fri Jan 25 16:27:38 +0000 

KeyboardInterrupt: 

> ### Exercise 1.1: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library display the list of all your friends (first 200) and sort them by its followers amounts in descending order. Record a sorted list of names in the `friends` dictionaries containing keys "name" and "followers_count" with respective data.

> #### HINT: `GET friends/list`  will be helpfull for you here.

In [14]:
# type your code here
url = 'https://api.twitter.com/1.1/friends/list.json'
params = {
    "q": 'python OR IPython',
    "count": 5, 
    "show_user": False,
    "geocode": '51.509865,-0.118092,20mi',
    "lang": 'en'
}
res = requests.get(url_3, auth=auth)

print res, res.status_code, res.headers['content-type']
print res.url

friends = res.json()
keys = []
values = []
for num, friends in enumerate(friends['users']):
    print 'My friend #{0}\t{1}'.format(num+1, friends['name'].encode("utf-8"))
    keys.append(friends['name'].encode("utf-8"))
    values.append(friends['followers_count'])

params = 0

friends = dict(zip(keys, values))
friends = sorted(friends.items(), key=lambda x: x[1], reverse=True)
friends

# url = ...
# params = ...

# friends = ...

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/friends/list.json
My friend #1	AI Memes for Artificially Intelligent Teens
My friend #2	John Papa
My friend #3	Twitter
My friend #4	Rasmus Bååth
My friend #5	The Lazy Goldmaker
My friend #6	Scott McMillan
My friend #7	Method
My friend #8	Ned Batchelder
My friend #9	Patrick Shyu
My friend #10	Kent Davis
My friend #11	angela he
My friend #12	Mikel Bober-Irizar
My friend #13	Ben Brode
My friend #14	+380991741601
My friend #15	WACKO
My friend #16	Andrej Karpathy
My friend #17	Chad Gardner
My friend #18	Tom Augspurger
My friend #19	Wes McKinney
My friend #20	Inside Amazon


[('Twitter', 55911244),
 ('Andrej Karpathy', 185994),
 ('Ben Brode', 168051),
 ('Method', 114474),
 ('John Papa', 79834),
 ('Scott McMillan', 67409),
 ('Wes McKinney', 40780),
 ('Inside Amazon', 13838),
 ('angela he', 11262),
 ('Kent Davis', 9766),
 ('Ned Batchelder', 8408),
 ('Patrick Shyu', 6658),
 ('AI Memes for Artificially Intelligent Teens', 5224),
 ('Mikel Bober-Irizar', 4593),
 ('Rasmus B\xc3\xa5\xc3\xa5th', 4185),
 ('WACKO', 4102),
 ('Tom Augspurger', 3714),
 ('The Lazy Goldmaker', 2563),
 ('Chad Gardner', 233),
 ('+380991741601', 1)]

In [15]:
from test_helper import Test

Test.twitterFriendsList(friends, url, auth, params, 'Incorrect query', "Exercise 1.1 is successful")

1 test failed. Incorrect query


> ### Exercise 1.2: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library display the list of 5 (or less if there is not so much) most recent tweets liked by you, which were also retweeted (`retweet_count` is not equal to zero). You shoud save the date of creation, author of the tweet, its text and retweets amount to the list `result` contains dictionaries with keys `"created_at"`, `"author"` and `"text"`, `"retweet_count"`.

> #### HINT: `GET favorites/list` will be helpfull for you here

In [18]:
# type your code here
url = 'https://api.twitter.com/1.1/favorites/list.json'
res = requests.get(url, auth=auth)

print res, res.status_code, res.headers['content-type']
print res.url

friends = res.json()
keys = []
values = []
keys1 = []
values1 = []
for num, friends in enumerate(friends):
    #print 'My likes #{0}\t{1}'.format(num+1, friends)
    keys.append(friends['created_at'].encode("utf-8"))
    values.append(friends['retweet_count'])
    keys1.append(friends['user']['screen_name'])
    values1.append(friends['text'].encode("utf-8"))
import pandas as pd
params = pd.DataFrame({"created_at":keys,
         "author":keys1,
         "text":values1,
         "retweet_count":values})
result = params[:5]
result.head(5)
# url = ...
# params = ...

# result = ...

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/favorites/list.json


,author,created_at,retweet_count,text
0,TheRealStanLee,Wed Nov 14 17:04:31 +0000 2018,114208,So many wonderful moments with Stan came spont...
1,Methodgg,Tue Nov 13 08:01:00 +0000 2018,143,Where were you when Arthas unleashed the Scour...
2,Warcraft,Wed Sep 19 18:47:25 +0000 2018,3041,Congratulations @Methodgg on the world first c...
3,WatcherDev,Wed Sep 19 18:27:55 +0000 2018,561,Congrats to @Methodgg on an amazing run throug...
4,CyberpunkGame,Sun Jun 10 21:40:18 +0000 2018,43622,&gt; Fetching file: cyberpunk2077_trailer_e3_2...


In [45]:
Test.twitterRecentTweets(result, url, auth, params, 'Incorrect query', "Exercise 1.2 is successful")

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

> ### Exercise 1.3: 

> Using [Twitter Streaming API](https://dev.twitter.com/streaming/overview) and `requests` Python library collect to the `last_tweets` Python list all tweets with hashtags _#twitter_, _#tweet_ and _#world_ that where appearing over 5 minutes (after that you ran a stream). `last_tweets` should contain 5 nested list each of which corresponds to tweets appeard over one minute along these 5 minutes (i.e. `last_tweets` should have form `[["tweets", "per", "the", "first", "minute"], ["tweets", "per", "the", "second", "minute"], ... ]`. Count also how many tweets written on English or written by user who has over 1000 follower were appearing each minute and write these 5 numbers to the `count5` Python tuple. 

> #### HINT: `POST statuses/filter` will be helpfull for you here

In [57]:
# type your code here
import time

url_5 = 'https://stream.twitter.com/1.1/statuses/filter.json?track=q'
params = {
    "q": 'twitter OR tweet OR world',
    #"count": 5, 
    #"show_user": False,
    #"geocode": '51.509865,-0.118092,20mi',
    #"lang": 'en'
}
res = requests.get(url_5, auth=auth, stream=True)

print res, res.status_code, res.headers['content-type']
print res.url, "\n"

# Let's measure the time elapsed after the start of streaming 
start = time.time()
l1,l2,l3,l4,l5 = [],[],[],[],[]
for line in res.iter_lines():
    # filter out keep-alive new lines
    if line:
        parsed_line = json.loads(line)
        print "Elapsed:", time.time()-start, "sec", parsed_line["created_at"], "\n", parsed_line['text'].encode("utf-8"), "\n"
        cap = int(time.time()-start)
        print cap
        if cap<60:
            l1.append(parsed_line['text'].encode("utf-8"))
        if cap>60:
            l1.append(parsed_line['text'].encode("utf-8"))
        if cap<120:
            l1.append(parsed_line['text'].encode("utf-8"))
        if cap>180:
            l1.append(parsed_line['text'].encode("utf-8")) 
        if cap>240:
            l1.append(parsed_line['text'].encode("utf-8"))
        if cap>10:
            break
last_tweets=[]
last_tweets.append(l1)
last_tweets.append(l2)
last_tweets.append(l3)
last_tweets.append(l4)
last_tweets.append(l5)
last_tweets

# url = ...

# last_tweets = ...

# count5 = ...

<Response [420]> 420 text/html
https://stream.twitter.com/1.1/statuses/filter.json?track=oathbringer 



ValueError: No JSON object could be decoded

In [58]:
Test.twitterHashtagsTweets(last_tweets, url, 'Incorrect content of "last_tweets" list', "Exercise 1.3.1 is successful")
Test.twitterHashtagsTweetsCount(count5, last_tweets, url, 'Incorrect content of "count5" list', "Exercise 1.3.2 is successful")

1 test failed. Incorrect URL


NameError: name 'count5' is not defined

---
## 2. Getting access to Twitter APIs using specific Python libraries
---

Today there are a few different Python specilized libraries which simplify our work with Twitter API - `tweepy`, `python-twitter`, `twython` ... (here we have called the most popular ones). Further we will consider one of them - `tweepy`.

### Installing `tweepy`
Let's install `tweepy` specific Python library which we will use work with Twitter APIs. 
The most straightforward way is through the `pip` installation tool. `Python >= 2.7.9` should come installed with pip, but for earlier versions, [see this guide for installing pip](https://pip.pypa.io/en/latest/installing.html).  
This can be run from the command line using:

    pip install tweepy
    
Make sure you upgrade pip to the newest version:

    pip install --upgrade pip

In [19]:
import tweepy

# Let's define consumer and access keys and secrets for getting access to Twitter API through your application
#consumer_key = '<YOUR CONSUMER KEY>'
#consumer_secret = '<YOUR CONSUMER SECRET CODE>'

#access_token = '<YOUR ACCESS TOKEN>'
#access_secret = '<YOUR ACCESS SECRET TOKEN>'

# Then you need authenticate yourself
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)
print api

In [20]:
# If the authentication was successful, you should see the name of the account print out
print "My name is", api.me().name

My name is Dmitriy Kisil


### 2.1: Read Tweets Appearing on Homepage

In [21]:
# Let's display first 3 tweets in your timeline (of course, if they are present there)
for (idx, tweet) in enumerate(tweepy.Cursor(api.home_timeline).items(3)):    
    print 'TWEET {0}:\n\t{1}\n'.format(idx, tweet.text.encode("utf-8"))

TWEET 0:
	RT @PleaseBeGneiss: Stranger asks you what time it is = kinda annoying

Stranger asks you what year it is = pretty concerning 

Stranger as…

TWEET 1:
	RT @AshleyS89102847: I'm beyond excited and proud to say I did my first coding project with @freeCodeCamp!!!! The feeling of accomplishment…

TWEET 2:
	PM дайджест #16: разбираемся с OKR, как вернуть Scrum-команде вдохновение, метрики в разработке… https://t.co/t6QrF63bTU



Let's look at the whole data contained in one tweet. Note, the `tweepy` api has decoded the JSON and put it into a more pythonic object. So for example, we can access the message in the tweet via python attribute access. 

In [22]:
import json

tweet = api.home_timeline()[0]
print type(tweet), '\n'
print tweet.text, '\n'
print json.dumps(tweet._json)

<class 'tweepy.models.Status'> 

RT @PleaseBeGneiss: Stranger asks you what time it is = kinda annoying

Stranger asks you what year it is = pretty concerning 

Stranger as… 

{"contributors": null, "truncated": false, "text": "RT @PleaseBeGneiss: Stranger asks you what time it is = kinda annoying\n\nStranger asks you what year it is = pretty concerning \n\nStranger as\u2026", "is_quote_status": false, "in_reply_to_status_id": null, "id": 1089112528091824129, "favorite_count": 0, "source": "<a href=\"https://tapbots.com/software/tweetbot/mac\" rel=\"nofollow\">Tweetbot for Mac</a>", "retweeted": false, "coordinates": null, "entities": {"symbols": [], "user_mentions": [{"id": 830468488967843840, "indices": [3, 18], "id_str": "830468488967843840", "screen_name": "PleaseBeGneiss", "name": "clean slate"}], "hashtags": [], "urls": []}, "in_reply_to_screen_name": null, "in_reply_to_user_id": null, "retweet_count": 1321, "id_str": "1089112528091824129", "favorited": false, "retweeted_status":

Let's see attributes associated with a Status object, which corresponds to any tweet.

In [23]:
tweet.__dict__.keys()

['contributors',
 'truncated',
 'text',
 'is_quote_status',
 'in_reply_to_status_id',
 'id',
 'favorite_count',
 '_api',
 'author',
 '_json',
 'coordinates',
 'entities',
 'in_reply_to_screen_name',
 'id_str',
 'retweet_count',
 'in_reply_to_user_id',
 'favorited',
 'retweeted_status',
 'source_url',
 'user',
 'geo',
 'in_reply_to_user_id_str',
 'lang',
 'created_at',
 'in_reply_to_status_id_str',
 'place',
 'source',
 'retweeted']

### 2.2: Get Twitter User Info

In [64]:
# We have chosen a random user
user = api.get_user(1088398616)
print "User name:", user.name, '\n'
print user

User name: saimadhu 

User(follow_request_sent=False, has_extended_profile=True, profile_use_background_image=False, _json={u'follow_request_sent': False, u'has_extended_profile': True, u'profile_use_background_image': False, u'profile_text_color': u'000000', u'default_profile_image': False, u'id': 1088398616, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png', u'verified': False, u'translator_type': u'none', u'profile_location': None, u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/808001022081671168/vLU7EBkK_normal.jpg', u'profile_sidebar_fill_color': u'000000', u'entities': {u'url': {u'urls': [{u'url': u'http://t.co/XWKZub79TD', u'indices': [0, 22], u'expanded_url': u'http://dataaspirant.com', u'display_url': u'dataaspirant.com'}]}, u'description': {u'urls': []}}, u'followers_count': 255, u'profile_sidebar_border_color': u'000000', u'id_str': u'1088398616', u'profile_background_color': u'000000', u'listed_count': 15, u'stat

### 2.3: Get a list of all your friends

In [24]:
for user in tweepy.Cursor(api.friends, screen_name="twitter").items():
    print user.screen_name

TwitterTogether
TwitterMediaJP
TwitterSeguro
AdsAPI
pichette
TwitterA11y
TwitterComms
TwitterSportsCA
TwitterVideoIN
TwitterREW
TwitterAdsHelp
TwitterMktgFR
JoinTheFlockJP
TwitterMktgBR
TwitterSportsJP
TwitterDevJP
TwitterTVJP
TwitterMedia
PeriscopeHelp
TwitterLifeline
TwitterSportsAU
TwitterNewsJP
TwitterGovJP
TwitterMusicJP
TwitterMktgMENA
momentsmena
TwitterAsians
TwitterMktgKR
UKMoments
CanadaMoments
MomentsBrasil
MomentsAU
MomentsES
MomentsJapan
kayvz
TwitterU
JoinTheFlockEU
verified
TwitterStripes
PeriscopeTV
TwitterParents
btaylor
IamDebraLee
Marthalanefox
TwitterDetroit
TwitterParis
leslieberland
TwitterPH
TwitterVideo
TwitterGaming
omidkordestani
TwitterMoments
TwitterATL
TwitterDUX
TwitterWomen
TwitterUK
TwitterTurkiye
TwitterEspana
TwitterSG
TwitterSeattle
TwitterOpen
TwitterMENA
TwitterLA
TwitterJP
TwitterID
TwitterFaith
TwitterDublin
TwitterCanada
TwitterBoulder
TwitterBoston
TwitterAlas
research
PeriscopeCo
NeighborNest
mopub
JoinNiche
Gnip
OfficialPartner
Blackbirds
mash

### 2.4: Search

Search is implemented directly through `tweepy.api`. As before let's search for a single tweet about "luck", phrased as a question. But we don't need worry now about URL characters encoding ☺

In [25]:
results = api.search(q='luck?', count=1)
print type(results)
print u'Created at: {0}\n"{1}"\nby {2}'.format(results[0].created_at, results[0].text, results[0].author.name)

<class 'tweepy.models.SearchResults'>
Created at: 2019-01-26 11:36:09
"With @Tama_Tonga seeing the error of his ways and now being the “Good Guy” can we see “Good Luck” Fale sometime soon pls? @TOKSFALE"
by Jordan Burn


You may create very specialized and more concrete queries. Let's find 5 tweets that contains the word "python" or "IPython" near London. We can provide this as a `geocode` with a lattitude, longitude and radius. We can also specify time range of tweets appearance (with the help of `since` and `until` key words) and the tweet language (the `lang` parameter restricts tweets to the given language).  

In [26]:
for tweet in api.search(q='python OR ipython', count=5, show_user=False,
                        geocode='51.509865,-0.118092,20mi', lang='en'):
    print tweet.created_at, '\n',  tweet.text, '\n'

2019-01-26 11:36:35 
RT @BAFTA: We're here to sprinkle a bit of fun over your weekend as we look back at Monty Python's hilarious Outstanding Contribution To Br… 

2019-01-26 11:33:04 
RT @RyoIndie: Just performance testing today. Python's giving me issues but we can work on that. #coding #python #pygame #programming  #sou… 

2019-01-26 11:32:48 
RT @BAFTA: We're here to sprinkle a bit of fun over your weekend as we look back at Monty Python's hilarious Outstanding Contribution To Br… 

2019-01-26 11:24:01 
JOB ALERT! Lead Python Software Engineer. Greenfields Project. London. £75,000/£80,000 - Since this is a new team,… https://t.co/nY3exUSxjG 

2019-01-26 11:23:39 
RT @BAFTA: We're here to sprinkle a bit of fun over your weekend as we look back at Monty Python's hilarious Outstanding Contribution To Br… 



### 2.5: Finding what are in the trend somewhere

According to the [tweepy API](http://tweepy.readthedocs.org/en/v3.5.0/api.html), we can return the top trending topics for a specific location, where the location is a `WOEID (Yahoo Where on Earth ID)`. 

The `WOEID` is a unique identifier, similar to zipcodes, but that expand worldwide. For example, London has a `WOEID` of 44418. You can search for `WOEID`'s here: http://woeid.rosselliot.co.nz/.

Let's see the top trending topics in New York

In [28]:
top_trends = api.trends_place(id=44418)
top_trends

[{u'as_of': u'2019-01-26T11:39:06Z',
  u'created_at': u'2019-01-26T11:31:52Z',
  u'locations': [{u'name': u'London', u'woeid': 44418}],
  u'trends': [{u'name': u'Osaka',
    u'promoted_content': None,
    u'query': u'Osaka',
    u'tweet_volume': 119634,
    u'url': u'http://twitter.com/search?q=Osaka'},
   {u'name': u'#Caturday',
    u'promoted_content': None,
    u'query': u'%23Caturday',
    u'tweet_volume': None,
    u'url': u'http://twitter.com/search?q=%23Caturday'},
   {u'name': u'#AustraliaDay',
    u'promoted_content': None,
    u'query': u'%23AustraliaDay',
    u'tweet_volume': 21684,
    u'url': u'http://twitter.com/search?q=%23AustraliaDay'},
   {u'name': u'#SaturdayMorning',
    u'promoted_content': None,
    u'query': u'%23SaturdayMorning',
    u'tweet_volume': 10254,
    u'url': u'http://twitter.com/search?q=%23SaturdayMorning'},
   {u'name': u'Kvitova',
    u'promoted_content': None,
    u'query': u'Kvitova',
    u'tweet_volume': 45325,
    u'url': u'http://twitter.com/s

As you can see, there's alot of metadata that goes into even a simple tweet. Let's slightly prettify the output data

In [29]:
print 10, "trends"#len(top_trends[0]['trends']), "trends"
for i, trend in enumerate(top_trends[0]['trends'][:10]):
    print i, '-', trend['name'], trend['url']

10 trends
0 - Osaka http://twitter.com/search?q=Osaka
1 - #Caturday http://twitter.com/search?q=%23Caturday
2 - #AustraliaDay http://twitter.com/search?q=%23AustraliaDay
3 - #SaturdayMorning http://twitter.com/search?q=%23SaturdayMorning
4 - Kvitova http://twitter.com/search?q=Kvitova
5 - #SaturdayMotivation http://twitter.com/search?q=%23SaturdayMotivation
6 - #BigGardenBirdWatch http://twitter.com/search?q=%23BigGardenBirdWatch
7 - Soccer AM http://twitter.com/search?q=%22Soccer+AM%22
8 - Michel Legrand http://twitter.com/search?q=%22Michel+Legrand%22
9 - GAME DAY http://twitter.com/search?q=%22GAME+DAY%22


### 2.6: Streaming

Like before, in case we want to "keep the connection open", and gather all the upcoming tweets, the streaming API is what we need. We need to extend the `StreamListener()` to customise the way we process the incoming data. Depending on the search term, we can get tons of tweets within a few minutes.

A working example that gathers all the new tweets with the _#iphone_ and _#android_ hashtag:

In [70]:
#This cell doesn't have limits and doesn't stop automatically. You need to interrupt it by yourself.

import time

from tweepy.streaming import StreamListener
from tweepy import Stream

class MyListener(StreamListener):
     
    def __init__(self):
        self.counter = 0
        self.begin_time = time.time()
        
    def on_data(self, data):
        try:
            self.counter += 1
            data = json.loads(data)
            # we will cut the tweet text
            print "Tweet #{0}: Created at {1} \n {2} \nTime elapsed from the beginning: {3} sec \n".\
                  format(self.counter, data['created_at'], data['text'][:100].encode('utf-8'), time.time()-self.begin_time)
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#iphone', '#android'])

Tweet #1: Created at Fri Jan 25 18:23:00 +0000 2019 
 "Android Clean Architecture with Kotlin, RxJava and Dagger 2" /by Антон Махров https://t.co/eRK8vA4e 
Time elapsed from the beginning: 2.08318090439 sec 

Tweet #2: Created at Fri Jan 25 18:23:02 +0000 2019 
 RT @DigiSlice: 💧[LEAKED] Alleged high-quality images of Samsung’s upcoming #GalaxyS10 and S10+ (Cred 
Time elapsed from the beginning: 3.72712993622 sec 

Tweet #3: Created at Fri Jan 25 18:23:04 +0000 2019 
 RT @ittablet: 🚨ATTENTION!!! COMPETITION!!!🚨

Win our award winning 10" Android tablet! 

Simply foll 
Time elapsed from the beginning: 5.92657709122 sec 



KeyboardInterrupt: 

> ### Exercise 1.4:

> Using `tweepy` Python library find when [Bill Gates](https://twitter.com/BillGates?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor) created his twitter account and what text had his last tweet.
Write the date and tweet text to the `result` dictionary containing keys `created_at` and `last_tweet_text`.

In [57]:
# type your code here
user = api.get_user(screen_name='BillGates')
print(user)
l1 =[1,2]
l1 = ["Wed Jun 24 18:44:10 +0000 2009",user.status.text.encode("utf-8")]

l2 = ["created_at","last_tweet_text"]
result = dict(zip(l2,l1))
result

User(follow_request_sent=False, has_extended_profile=False, profile_use_background_image=False, _json={u'follow_request_sent': False, u'has_extended_profile': False, u'profile_use_background_image': False, u'profile_text_color': u'333333', u'default_profile_image': False, u'id': 50393960, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png', u'verified': True, u'translator_type': u'regular', u'profile_location': None, u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/988775660163252226/XpgonN0X_normal.jpg', u'profile_sidebar_fill_color': u'DDEEF6', u'entities': {u'url': {u'urls': [{u'url': u'https://t.co/qXMN8oq38K', u'indices': [0, 23], u'expanded_url': u'https://www.gatesnotes.com/', u'display_url': u'gatesnotes.com'}]}, u'description': {u'urls': []}}, u'followers_count': 46324834, u'profile_sidebar_border_color': u'000000', u'id_str': u'50393960', u'profile_background_color': u'C0DEED', u'listed_count': 122330, u'status': {u'co

{'created_at': 'Wed Jun 24 18:44:10 +0000 2009',
 'last_tweet_text': 'I look at a lot of global health data, and one trend stands above the rest: Global health groups like @gavi,\xe2\x80\xa6 https://t.co/0qoSHpZlQ8'}

In [58]:
Test.twitterBillGates(result, api, 'Incorrect query', "Exercise 1.4 is successful")

1 test failed. Incorrect query


<center><h3>Presented by <a target="_blank" rel="noopener noreferrer nofollow" href="http://datascience-school.com">datascience-school.com</a></h3></center>